# Operator Overloading Playground!

In [ ]:
%load_ext cython

## 1. Error propagation (found online)

Imagine you have data with error bounds $x = 20 ± 0.1$ and $y = 30 ± 0.2$.  What is the result of $x + y$?

For addition or subtraction, if

$$
Q = a + b - c - d
$$

then if the error in $a$ is $\delta a$, then the error in $Q$ is

$$
\delta Q = \sqrt{ (\delta a)^2 + (\delta b)^2 +(\delta c)^2 +(\delta c)^2}
$$

In [ ]:
%%cython
from libc.math cimport sqrt

cdef class Unsure:
    cdef double value
    cdef double error
    def __init__(self, double value, double error=0):
        self.value = value
        self.error = error
        
    def __add__(x, y):
        if not isinstance(x, Unsure) and not isinstance(y, Unsure):
            raise Exception('Bad Types')
    
        cdef Unsure px = <Unsure>x
        cdef Unsure py = <Unsure>y
        
        cdef Unsure result = Unsure(0)
        result.value = px.value + py.value
        result.error = sqrt(px.error**2 + py.error**2)
        return result
    
    def __repr__(self):
        return '{:.6g} ± {:.3g}'.format(self.value, self.error)

In [ ]:
x = Unsure(21.5, 0.5)
print(x)

In [ ]:
y = Unsure(47.5, 0.5)
print(y)

In [ ]:
x + y

## 2. Interval Arithmetic (found online)

In interval arithmetic, each number is really an interval with an upper and lower bound.  When calculations with such numbers are performed, we always make sure the answer retains the correct upper and lower bound.

For example, if 

$$
x = \left[ x_{min} \;\;, x_{max} \;\; \right]
$$

And similar for $y$, then

$$
x + y = [ x_{min} \;\; + y_{min} \;\;, x_{max} \;\; + y_{max} \;\;]
$$


In [ ]:
%%cython
cimport cython

@cython.freelist(10)
cdef class Interval:
    cdef double vmin, vmax
    def __init__(self, double vmin, double vmax):
        self.vmin = vmin
        self.vmax = vmax
        
    def __add__(x, y):
        if not isinstance(x, Interval) and not isinstance(y, Interval):
            raise Exception('Bad Types')
    
        cdef Interval px = <Interval>x
        cdef Interval py = <Interval>y
        
        cdef Interval result = Interval(0, 0)
        result.vmin = px.vmin + py.vmin
        result.vmax = px.vmax + py.vmax
        return result
    
    def __repr__(self):
        return '[{:.6g}, {:.6g}]'.format(self.vmin, self.vmax)

In [ ]:
x = Interval(21.0, 22.0)
print(x)

In [ ]:
y = Interval(47.0, 48.0)
print(y)

In [ ]:
def my_calculation(a, b):
    return a + b

print(my_calculation(x, y))
print(my_calculation(10, 20))

## 3. Dependency Tracking

It would be very cool to have a bunch of calculations where _afterwards_ you can tell which values depend on which other values.

In [ ]:
%%cython
import numbers
from weakref import WeakSet  # WeakSet, so intermediates get deleted

cdef class Tracked:
    cdef object __weakref__
    cdef double value
    cdef public str name
    cdef public object depends_on  # This value depends on others
    def __init__(self, double value, name=None):
        self.value = value
        self.name = name
        self.depends_on = WeakSet() 
    
    @staticmethod
    def handle_type(x):
        if isinstance(x, numbers.Number): 
            return Tracked(x)
        elif isinstance(x, Tracked):
            return x
        else:
            raise Exception('Bad Types')
        
    def __mul__(x, y):
        cdef Tracked px = <Tracked>Tracked.handle_type(x)
        cdef Tracked py = <Tracked>Tracked.handle_type(y)
        
        cdef Tracked result = Tracked(0)
        
        result.value = px.value * py.value
        result.name = '(calculated)'.format(px.name, py.name)
        
        result.depends_on.add(px) 
        result.depends_on.add(py)
        result.depends_on |= px.depends_on | py.depends_on
        
        return result
            
    def __repr__(self): 
        depon = [' '*8+x.name for x in self.depends_on]
        s = '{:.6g} name="{}"'.format(self.value, self.name)
        if depon:
            s += '\n    I depend on: \n{}'.format('\n'.join(depon))
        return s

In [ ]:
ww = Tracked(0.2, name='wall width, metres, measured 5 May 1995 by CJRH')
print(ww)

In [ ]:
wh = Tracked(2.0, name='wall height, metres, measured 12 Jan 1998 by XYZ')
print(wh)

wl = Tracked(5.0, name='wall length, metres, ESTIMATE sometime by ABC')
print(wl)

In [ ]:
def volume(width, height, length):
    return width * height * length

print(volume(ww, wh, wl))

In [ ]:
def mass(volume, density):
    return volume * density



d = Tracked(2000.0, 'brick (kg/m3), https://en.wikipedia.org/wiki/Brick')
print(
    mass(volume(ww, wh, wl), d)
)

## Units of measurement

See https://github.com/cjrh/misu

In [ ]:
!pip install misu

In [ ]:
from misu import *

In [ ]:
x = 100*kg
y = 300*lb

z = x + y
print(z)

In [ ]:
z >> lb

In [ ]:
z + 100*stone

In [ ]:
z + 200*feet